In [1]:
import aiohttp
import asyncio
import pandas as pd
from dotenv import load_dotenv
import os
import numpy as np

In [2]:
load_dotenv()
read_access = os.getenv("Read_Access_Token")
API = os.getenv("API")

In [3]:
df = pd.read_csv("./movies.csv")

In [4]:
ids = np.array(df['id'])
ids = ids.astype(str)


In [5]:
async def gather_calls(movie_id,session):
    header = {
        'Authorization': f"Bearer {read_access}",
        'accept': "application/json"
    }
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/external_ids"
    async with session.get(url,headers=header) as response:
        try:
            if response.status == 200:
                data = await response.json()
                return pd.DataFrame([data])
            return pd.DataFrame()
        except Exception as e:
            print(f"{e} on movieid {movie_id}")
            return pd.DataFrame()

In [6]:
async def fetch_all_data(movie_ids):
    async with aiohttp.ClientSession() as session:
        tasks = list()
        for movie_id in movie_ids:
            tasks.append(asyncio.create_task(gather_calls(movie_id,session)))
        results = await asyncio.gather(*tasks)
        return pd.concat(results,ignore_index=True)


In [7]:
data = await fetch_all_data(ids)

In [8]:
data.to_csv("movies_id.csv")